In [31]:
""" Define imports to work with PySpark """

import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StringType

In [3]:
""" Configure environment """ 

os.environ["SPARK_HOME"] = "/usr/local/bin/spark-3.5.1-bin-hadoop3"
spark_session = SparkSession.builder.appName("Select records").getOrCreate()
input_file_path = "/Users/fernandoferreira/Code/pyspark-course/data/"

24/12/29 12:10:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/12/29 12:10:33 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
""" Load data to spark data frame through 'generic' read method """

empresas_database_file = f"{input_file_path}/empresas"
empresas_df = spark_session.read.format('csv').load(empresas_database_file, sep=';', inferSchema=True)

In [11]:
""" Rename Columns """
empresas_column_name = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

for index, column in enumerate(empresas_column_name):
    empresas_df = empresas_df.withColumnRenamed(f"_c{index}", column)

In [13]:
""" Select all coluns from dataframe """
empresas_df\
    .select('*')\
    .show(5, False)

+-----------+-------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial                                                        |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|403        |ALUPE REPRESENTACOES LTDA                                                            |2240             |49                         |0,00                     |1               |NULL                       |
|2498       |JCA COMERCIO E REPRESENTACOES LTDA                                                   |2062             |49             

In [15]:
""" Select specific columns from dataframe """

empresas_df.select('natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa').show(5, False)

+-----------------+----------------+-------------------------+
|natureza_juridica|porte_da_empresa|capital_social_da_empresa|
+-----------------+----------------+-------------------------+
|2240             |1               |0,00                     |
|2062             |5               |0,00                     |
|2135             |5               |0,00                     |
|2135             |1               |0,00                     |
|2224             |5               |0,00                     |
+-----------------+----------------+-------------------------+
only showing top 5 rows



In [17]:
" Load Socios database files "

socios_database_files = f"{input_file_path}/socios"
socios_df = spark_session.read.format('csv').load(socios_database_files, sep=';', inferShema=True)

In [18]:
" Rename Socios dataframe columns "
socios_column_names = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

for index, column in enumerate(socios_column_names):
    socios_df = socios_df.withColumnRenamed(f"_c{index}", column)

print("Socios dataframe columns renamend successfully")

Socios dataframe columns renamend successfully


In [23]:
" Select specific columns in row format "
socios_df\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', 'data_de_entrada_sociedade')\
    .show(5, False)

+-------------------------------+------------+-------------------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|data_de_entrada_sociedade|
+-------------------------------+------------+-------------------------+
|LILIANA PATRICIA GUASTAVINO    |7           |19940725                 |
|CRISTINA HUNDERTMARK           |7           |19940725                 |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |19940516                 |
|EDUARDO BERRINGER STEPHAN      |5           |19940516                 |
|HANNE MAHFOUD FADEL            |8           |19940609                 |
+-------------------------------+------------+-------------------------+
only showing top 5 rows



In [27]:
" Select specific columns formating date column "
socios_df\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .show(5, False)


+-------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|ano_de_entrada|
+-------------------------------+------------+--------------+
|LILIANA PATRICIA GUASTAVINO    |7           |NULL          |
|CRISTINA HUNDERTMARK           |7           |NULL          |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |NULL          |
|EDUARDO BERRINGER STEPHAN      |5           |NULL          |
|HANNE MAHFOUD FADEL            |8           |NULL          |
+-------------------------------+------------+--------------+
only showing top 5 rows



In [29]:
" Print Schema of Socios Dataframe "
socios_df.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- identificador_de_socio: string (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: string (nullable = true)
 |-- data_de_entrada_sociedade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: string (nullable = true)
 |-- faixa_etaria: string (nullable = true)



In [32]:
" Casting date column to date format in order to use specific functions to handle date "

socios_df = socios_df.withColumn('data_de_entrada_sociedade', f.to_date(socios_df.data_de_entrada_sociedade.cast(StringType()), 'yyyyMMdd'))

In [33]:
" Print Schema of Socios Dataframe after change colum type "

socios_df.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- identificador_de_socio: string (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: string (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: string (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: string (nullable = true)
 |-- faixa_etaria: string (nullable = true)



In [34]:
" Select specific columns formating date column after casting data_de_entrada_sociedade column to date"
socios_df\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .show(5, False)


+-------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|ano_de_entrada|
+-------------------------------+------------+--------------+
|LILIANA PATRICIA GUASTAVINO    |7           |1994          |
|CRISTINA HUNDERTMARK           |7           |1994          |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |1994          |
|EDUARDO BERRINGER STEPHAN      |5           |1994          |
|HANNE MAHFOUD FADEL            |8           |1994          |
+-------------------------------+------------+--------------+
only showing top 5 rows



In [49]:
" Load Estabelecimentos Dataframe "

estabelecimentos_database_files = f"{input_file_path}/estabelecimentos"

estabelecimentos_df = spark_session.read.format('csv').load(estabelecimentos_database_files, sep=';', inferSchema=True)

In [64]:
" Handle Estabelecimentos Dataframe "

estabelecimento_column_names = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
for index, column_name in enumerate(estabelecimento_column_names):
    estabelecimentos_df = estabelecimentos_df.withColumnRenamed(f"_c{index}", column_name)

estabelecimentos_df = estabelecimentos_df\
                        .withColumn("data_de_inicio_atividade", 
                                    f.to_date(estabelecimentos_df.data_de_inicio_atividade.cast(StringType()), "yyyyMMdd"))

In [65]:
estabelecimentos_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string

In [71]:
estabelecimentos_df\
    .select("nome_fantasia", \
            "municipio", \
            f.year('data_de_inicio_atividade').alias('ano_de_inicio_atividade'),\
            f.month('data_de_inicio_atividade').alias('mes_de_inicio_atividade'))\
    .show(5, False)

+-----------------+---------+-----------------------+-----------------------+
|nome_fantasia    |municipio|ano_de_inicio_atividade|mes_de_inicio_atividade|
+-----------------+---------+-----------------------+-----------------------+
|PIRAMIDE M. C.   |7107     |1994                   |5                      |
|NULL             |7107     |1994                   |5                      |
|NULL             |7107     |1994                   |5                      |
|NULL             |7107     |1994                   |5                      |
|EMBROIDERY & GIFT|7075     |1995                   |5                      |
+-----------------+---------+-----------------------+-----------------------+
only showing top 5 rows

